In [23]:
library(glmnet)
library(doMC)
library(survival)
library(data.table)
library(mltools)
library(CoxBoost)
library(randomForestSRC)
library(CoxHD)
source('run_prognosis.R')

In [13]:
df_all <-read.table("df_prognosis.tsv",sep = '\t' , header = T) 

###    2094 rows 166 columns

df_all <-na.omit(df_all) # delete rows with na (161)
df_all <- df_all[df_all$os>0,] # delete when os is negative (2)

####

#Convert predicted_component to one hot encoder
df_all$new_eln<-factor(df_all$new_eln, levels = c("adverse","intermediate","favorable"), labels = 0:2, ordered = TRUE)  # convert categorical new_eln to numerical (0,1,2)
name <-rownames(df_all)
df_all$predicted_component <- as.factor(df_all$predicted_component)
df_final <- as.data.frame(one_hot(as.data.table(df_all),cols="predicted_component"))
rownames(df_final) <- name

#write.table(df_final,"df_prognosis_features_ready.tsv",sep='\t',quote=F)
####

###  1931 rows 180 columns

In [29]:
#dim(df_all)
#dim(df_final)
df_final <- read.table("df_prognosis_features_ready.tsv",sep = '\t' , header = T)
x <- data.matrix(df_final[,1:84])
y <- data.matrix(df_final[,c("os","os_status")])
predictors <- c(predictorGLM)
#predictors <- c(rep(list(predictorGLM),10),rep(list(predictorRF),10),predictorBoost,predictorRFX)
str_predictors <-c(rep("CoxMod",10),rep("RFS",10),"CoxBoost","RFX")
l_alpha <-seq(0.1,1,0.1)
l_ntree <- seq(100,1000,100)
launch_prognosis(x,y,predictors,str_predictors,l_alpha,l_ntree)

[1] "start CV"


CoxMod_
0.6196853
0.6371083
0.6375430
0.6515947
0.6566266
0.6671471
0.6357661
0.6370673
0.6466425
0.6198613


In [14]:
x <- data.matrix(df_final[,1:84])
y <- data.matrix(df_final[,c("os","os_status")])
predictors <- c(rep(list(predictorGLM),2),rep(list(predictorRF),2),predictorBoost,predictorRFX,predictorCPSS)
#predictors <- c(rep(list(predictorGLM),10),rep(list(predictorRF),10),predictorBoost,predictorRFX)
str_predictors <-c(rep("CoxMod",10),rep("RFS",10),"CoxBoost","RFX")
l_alpha <-seq(0.1,1,0.1)
l_ntree <- seq(100,1000,100)
launch_prognosis(x,y,predictors,str_predictors,l_alpha,l_ntree,mc.cores=1)

[1] "start CV"
[1] "start CV"
[1] "start CV"
[1] "start CV"
[1] "start CV"
[1] "start CV"
[1] "start CV"


CoxMod_0.1,CoxMod_0.2,CoxMod_100,CoxMod_200,CoxMod_,CoxMod_,CoxMod_
0.6265973,0.6264747,0.6299062,0.6263767,0.6294487,0.6266708,0.6196853
0.6511334,0.6518875,0.6502762,0.6506096,0.6514747,0.6557450,0.6371083
0.6376802,0.6371071,0.6475451,0.6435249,0.6387055,0.6461404,0.6375430
0.6470114,0.6483706,0.6514524,0.6541549,0.6513023,0.6550400,0.6515947
0.6554482,0.6555739,0.6509467,0.6528007,0.6540498,0.6634692,0.6566266
0.6808461,0.6810398,0.6645901,0.6640555,0.6803115,0.6784054,0.6671471
0.6326469,0.6329174,0.6390286,0.6392832,0.6339360,0.6346362,0.6357661
0.6370511,0.6383572,0.6368657,0.6363416,0.6393892,0.6390828,0.6370673
0.6442483,0.6457528,0.6492146,0.6455101,0.6483006,0.6560574,0.6466425
0.6153510,0.6145104,0.6357523,0.6376598,0.6155369,0.6197643,0.6198613


In [21]:
write.table(df_final,"features.tsv",col.names = NA,sep='\t',quote=F)

In [22]:
#read.table("features.tsv",sep='\t')

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181
,new_eln,ASXL1,ASXL2,ASXL3,ATRX,BAGE3,BCOR,BRAF,CBFB,⋯,bm_blasts,secondary,wbc,hb,plt,gender,age,os,os_status,cr
PD14868a,1,0,0,0,0,0,0,0,0,⋯,89,1,175,9.8999939,199,0,79.7125256673511,0.191649555099247,1,0
PD14869c,1,0,0,0,0,0,0,0,0,⋯,90,1,3.1,8.6999969,23,1,65.0431211498973,4.70636550308008,0,1
PD14871a,1,0,0,0,0,0,0,0,0,⋯,62,1,127,7.5999985,57,1,79.6577686516085,0.37782340862423,1,1
PD14872a,0,0,0,0,0,0,1,0,0,⋯,40,1,8.2,7.6999969,124,0,66.5051334702259,0.0492813141683778,1,0
PD14873a,0,1,0,0,0,0,0,0,0,⋯,25,2,44.2,9.2999954,40,1,71.7809719370294,0.550308008213552,1,0
PD14874a,0,0,0,0,0,0,0,0,0,⋯,56,1,69,9.0999985,63,1,75.0143737166324,0.20807665982204,1,0
PD14875a,2,0,0,0,0,0,0,0,0,⋯,70,1,43.4,10.299995,64,0,74.7953456536619,1.58795345653662,1,1
PD14876c,0,0,0,0,0,0,0,0,0,⋯,90,2,59.1,7.5999985,104,1,65.6591375770021,0.243668720054757,1,0
PD14877c,2,0,0,0,0,0,0,0,0,⋯,92,1,144,6.2999992,47,0,84.104038329911,0.0657084188911704,1,0
